In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import os

Diabetic retinopathy, a complication associated with diabetes, affects the eys and can lead to blindness if not diagnosed. This condition rsults from damage to the blood vessels inside the retina. Diabetic retinopathy is a leading cause of blindness among adults.\n",
    "\n",
    "The goal of this project is to implement a machine learning model that can accurately predict the presence of diabetic retinopathy using retinal images.We will be using the [diabetic retinopathy](https://www.kaggle.com/datasets/tanlikesmath/diabetic-retinopathy-resized/) dataset from kaggle, consisting of over 35,000 1024x1024 retinal scans. Below is an example of an individual retinal scan.

In [ ]:
def plot_image(path):

    """ Plot a provided image """
    img = Image.open(path)
    
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    
plot_image('./dataset/resized_train_cropped/34035_right.jpeg')

We begin by preprocessing all images in the dataset. Normalization involves converting the each .jpeg file into an $m \times n$ array. The `normalize() function converts the elements of the array into 16 bit float point variables between the value of 0 and 1.

In [ ]:
def normalize(path):

    """ Normalize an image by resizing and scaling pixel values """

    # resize 1024x1024 image to 512x512
    img = Image.open(path).resize((512, 512))

    # set values as float16 between 0 and 1
    array = np.array(img).astype(np.float16) / 255.0

    return array

The function below uses a `ImageDataGenerator` to augment a single image per call to `augment()`. The goal of augmentation is to artificially increase diversity in a training set.

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,       # rotate image
    width_shift_range=0.2,   # shift width 
    height_shift_range=0.2,  # shift height 
    shear_range=0.2,         # shear image
    zoom_range=0.2,          # zoom in or out
    horizontal_flip=True,    # flip horizontal?
    fill_mode='nearest'      # fill new pixels
)

def augment(array):

    """ Augment image using ImageImageDataGenerator """

    # add dimension for IDG
    img = np.expand_dims(array, 0)

    # create iterator to perform augmentation
    it = datagen.flow(img, batch_size = 1)

    # retrieve image and conver to desired datatype
    augmented = next(it)[0].astype(np.float16)

    return augmented

The `process()` function just calls `augment()` and `normalize()` functions and returns an exception if anything is wrong.

In [ ]:
def process(path):

    """ Process singular image given path """

    try:
        # normalize and augment image
        array = normalize(path)
        array = augment(array)

        return array

    except Exception as e:
        print(f"Error processing {path}: {e}")
        return None

Below we load image data from a csv file, clean it to remove unecessary columns, and then prepare a dataset by appending the full path to each image - `image` column is added to list of images as `./dataset/image.jpeg`.

In [ ]:
labels_csv_path = './dataset/trainLabels_cropped.csv'
labels_df = pd.read_csv(labels_csv_path)

# remove useless columns
labels_df = labels_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])

# define image directory
image_dir = './dataset/resized_train_cropped/'

# append path and extension to each image name
labels_df['full_image_path'] = image_dir + labels_df['image'] + '.jpeg'

# split into training and validation (80% and 20% respectively)
train_indices, val_indices = train_test_split(labels_df.index, test_size=0.2, random_state=42)

# generate all image paths
train_image_paths = [labels_df.iloc[i]['full_image_path'] for i in train_indices]

The `load_data_gen` function is a generator that loads and processes images in batches from the provided paths from above. It pairs each image with its corresponding label and then creates label and image numpy arrays.

In [ ]:
def load_data_gen(image_paths, labels_df, batch_size=32):
    
    """ Generator to load and process data in batches """
    # total num of images
    num_samples = len(image_paths)
    
    while True: # loop while next image is True
        
        for offset in range(0, num_samples, batch_size):
            
            batch_images = [] # images in current batch
            batch_labels = [] # labels in current batch

            # iterate over batch
            for i in range(offset, min(offset + batch_size, num_samples)):
                
                path = image_paths[i] # get path of current image
                img = process(path)   # process current image

                # if image processing was successful, add to batch
                if img is not None:
                    batch_images.append(img)
                    # get label for current image
                    label = labels_df.iloc[i]['level']
                    batch_labels.append(label)

            # convert list of images and labels to numpy arrays
            batch_images_np = np.array(batch_images)
            batch_labels_np = np.array(batch_labels)
            
            # return images and labels 
            yield batch_images_np, batch_labels_np

We define a convolutional neural network for 512x512 pixel images with three color channels. The network uses convolutional layers to reduce the image dimensions, followed by flattening the multi-dimensional array into a one-dimensional array. This is then processed through dense layers, integrating the learned features, with a dropout layer included to help prevent overfitting. The network concludes with a final dense layer that outputs a probability score, indicating the classification of the image. The model is compiled with the Adam optimizer and binary cross-entropy loss, focusing on accuracy

In [ ]:
def build_model():
    
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
batch_size = 32
train_generator = load_data_gen(train_image_paths, labels_df, batch_size=batch_size)

# calculate num of steps per epoch
steps_per_epoch = len(train_indices) // batch_size

# start fitting the model using the training generator
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=10)